# Youtube Ranking

- 유투브 랭킹 - 전체

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 

In [3]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [5]:
channel = channel_list[0]
# 1위 블핑

In [6]:
rank = channel.select_one('.rank').get_text().strip()
rank = int(rank)
rank

1

In [7]:
category = channel.select_one('.category').get_text().strip()
category[1:-1]
# [는 인덱스 0취급이면서 왜 ]는 -1에 안 걸리지? 

'음악/댄스/가수'

In [8]:
name = channel.select_one('.subject a').get_text().strip()      # subject밑의 a태그 찾아주기
name

'BLACKPINK'

In [9]:
subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
view = channel.select_one('.view_cnt').get_text().strip()
video = channel.select_one('.video_cnt').get_text().strip()


subscriber, view, video 

('7020만', '214억5709만', '395개')

- 반복문으로 페이지 내 데이터 수집하기

In [10]:
lines = []
for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('.category').get_text().strip()
    name = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()

    lines.append([rank, category[1:-1], name, subscriber, view, video])

df = pd.DataFrame(lines, columns = ['순위','장르','채널명','구독자수','조회수','비디오수'])
df.head(10)
    

,순위,장르,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,7020만,214억5709만,395개
1,2,음악/댄스/가수,HYBE LABELS,6300만,204억9684만,725개
2,3,음악/댄스/가수,BANGTANTV,6150만,140억7794만,"1,646개"
3,4,음악/댄스/가수,SMTOWN,2930만,230억1641만,"3,779개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
5,6,음악/댄스/가수,1MILLION Dance Studio,2460만,69억3037만,"3,396개"
6,7,음악/댄스/가수,1theK (원더케이),2370만,213억5884만,"15,675개"
7,8,음악/댄스/가수,JYP Entertainment,2300만,150억0482만,"1,429개"
8,9,음악/댄스/가수,Mnet K-POP,1900만,139억7423만,"26,841개"
9,10,음악/댄스/가수,JFlaMusic,1740만,35억0198만,281개


- 숫자로 바꿔주는 함수 ('만' : 0000, 개, 억 없애기 등) 

In [11]:
def convert_unit(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만', '0000')
    return f'{int(s):,d}'        # 데시말? 천단위 구분기호 
    

In [12]:
convert_unit('123억5200만')

'12,352,000,000'

In [13]:
df['구독자수'] = df.구독자수.apply(convert_unit)
df['조회수'] = df.조회수.apply(convert_unit)
df['비디오수'] = df.비디오수.apply(convert_unit)
df.head()

,순위,장르,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"70,200,000","21,457,090,000",395
1,2,음악/댄스/가수,HYBE LABELS,"63,000,000","20,496,840,000",725
2,3,음악/댄스/가수,BANGTANTV,"61,500,000","14,077,940,000","1,646"
3,4,음악/댄스/가수,SMTOWN,"29,300,000","23,016,410,000","3,779"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


- 두번째 페이지

In [14]:
# Xpath로찾아가기 - //*[@id="list-skin"]/nav/span/a[1] : F12에서 2페이지 버튼 우클릭 - copy xpath 
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
time.sleep(2)

In [15]:
soup= BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)


100

In [16]:
lines = []
for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('.category').get_text().strip()
    name = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()

    lines.append([rank, category[1:-1], name, convert_unit(subscriber), convert_unit(view), convert_unit(video)])

In [17]:
df = pd.DataFrame(lines, columns = ['순위','장르','채널명','구독자수','조회수','비디오수'])
df.head(10)

,순위,장르,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"70,200,000","21,457,090,000",395
1,2,음악/댄스/가수,HYBE LABELS,"63,000,000","20,496,840,000",725
2,3,음악/댄스/가수,BANGTANTV,"61,500,000","14,077,940,000","1,646"
3,4,음악/댄스/가수,SMTOWN,"29,300,000","23,016,410,000","3,779"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
5,6,음악/댄스/가수,1MILLION Dance Studio,"24,600,000","6,930,370,000","3,396"
6,7,음악/댄스/가수,1theK (원더케이),"23,700,000","21,358,840,000","15,675"
7,8,음악/댄스/가수,JYP Entertainment,"23,000,000","15,004,820,000","1,429"
8,9,음악/댄스/가수,Mnet K-POP,"19,000,000","13,974,230,000","26,841"
9,10,음악/댄스/가수,JFlaMusic,"17,400,000","3,501,980,000",281


- page 1 ~ 10까지 크롤링하기

In [18]:
lines = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        rank = int(channel.select_one('.rank').get_text().strip())
        category = channel.select_one('.category').get_text().strip()
        name = channel.select_one('.subject a').get_text().strip()
        subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
        view = channel.select_one('.view_cnt').get_text().strip()
        video = channel.select_one('.video_cnt').get_text().strip()

        lines.append([rank, category[1:-1], name, convert_unit(subscriber), convert_unit(view), convert_unit(video)])
        

In [19]:
df = pd.DataFrame(lines, columns=['순위','장르','채널명','구독자수','조회수','비디오수'])
df.head()

,순위,장르,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"70,200,000","21,457,090,000",395
1,2,음악/댄스/가수,HYBE LABELS,"63,000,000","20,496,840,000",725
2,3,음악/댄스/가수,BANGTANTV,"61,500,000","14,077,940,000","1,646"
3,4,음악/댄스/가수,SMTOWN,"29,300,000","23,016,410,000","3,779"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [22]:
# chrome 페이지 10까지 돌아가는지 두눈으로 확인하세요! 

In [20]:
df.to_csv('유투브랭킹.csv', index = False)

In [21]:
driver.close()